In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

In [3]:
def compute_some_stats(stats):
    stats = np.matrix(stats)
    mean_x = np.mean(stats, axis=0)
    std_x = np.std(stats, axis = 0)
    
    return mean_x, std_x

In [4]:
def gillespie(x0, k, T, V,Vr):
    x=copy.deepcopy(x0)
    t=0
    statsX = [copy.deepcopy(x0)]
    
    while t <= T :
        
        lambd=[]
        for i in range(len(V[0])) :
            for j in range(len(V)): 
                tmp = k[i]
                if Vr[j][i]<0 : #c'est un réactif
                    tmp=tmp*x[j]
            lambd.append(tmp)
        Rtot=sum(lambd)
        deltat=float(np.random.exponential(size=1,scale=(1/Rtot)))
        lambd_norm=[lambd[i]/Rtot for i in range(len(lambd))]
        a=[i for i in range(len(V[0]))]
        for l in range(len(x)) : 
            x[l]=x[l]+V[l][np.random.choice(a,p=lambd_norm)]
        t = t+deltat
        
        statsX.append(copy.deepcopy(x))
        
        mean_x, std_x = compute_some_stats(statsX)
        
    return x, mean_x, std_x

In [5]:
k=[1,3]
x0=[2,1,2]
V=[[-1,1],[-1,1],[1,-1]]
Vr=V

#-1   1
#-1   1
# 1  -1


gillespie(x0,k,2,V,Vr)

([5, 2, 1],
 matrix([[4.16666667, 1.83333333, 1.16666667]]),
 matrix([[1.34370962, 0.68718427, 0.68718427]]))

Exercice 2

In [88]:
x0_ex2=[100,100,100,100,100]
T_ex2=2
#Vr : 
#   R1  R2  R3  R4  R5  R6  R7
# A 0   -1  -1  -1  -1  0   0
# B 0   0   -1   0  -1  0   0
# C 0   0    0   0  0   -1  -1
# D 0   0    0   0  0   0   0
# E 0   0    0   0  0   0   0

#V :  
#   R1  R2  R3  R4  R5  R6  R7
# A 1   -1  -1  0   -1  1   0
# B 0   0   -1  1   -1  1   0
# C 0   0    1  0    1  -1  -1
# D 0   0    0  0    0  0   1
# E 0   0    0  0    0  0   1

k1= 0.1
k2 = 0.05
k = 0.2
k_ex2=[k1, k2,k, k, k1, k1, k2]

V_ex2=[[1,-1,-1,0,-1,1,0],[0,0,-1,1,-1,1,0],[0,0,1,0,1,-1,-1], [0,0,0,0,0,0,1],[0,0,0,0,0,0,1]]
Vr_ex2=[[0,-1,-1,-1,-1,0,0],[0,0,-1,0,-1,0,0],[0,0,0,0,0,-1,-1], [0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]

gillespie(x0_ex2,k_ex2, 10, V_ex2, Vr_ex2)

([98, 99, 102, 101, 100],
 matrix([[ 98.625, 100.25 , 101.375, 100.25 , 100.   ]]),
 matrix([[0.99215674, 0.96824584, 0.69597055, 0.4330127 , 0.        ]]))

In [24]:
# Equations for the molecules (ODE) :

#R1 
# dAdt = k1

#R2 
# dAdt = -k2[A]

#R3
# dAdt = -k[A]
# dBdt = -k[B]
# dCdt = k[A] + k[B] = k[C]

#R4
# dAdt = -k[A] + k[A] = 0
# dBdt = k[A]

#R5
# dAdt = -k1[A]
# dBdt = -k1[B]
# dCdt = k1[A] + k1[B] = k1[C]

#R6
# dAdt = k1[C]
# dBdt = k1[C]
# dCdt = -k1[C]

#R7
# dCdt = -k2[C]
# dDdt = k2[C]
# dEdt = k2[C]


# Resulting of the 7 reactions :

# dAdt = k1 -k2[A] -k[A] -k1[A] + k1[C]
# dBdt = -k[B]+k[A]-k1[B]+k1[C]
# dCdt = k[C] + k1[C] -k1[C] - k2[C] 
# dDdt = k2[C]
# dEdt = k2[C]


In [15]:
from scipy.integrate import odeint 

In [61]:
def dX_dt(x,t, k1, k2, k):
    return (k1*1 - k2*x[0] - k*x[0] - k1*x[0] + k1*x[2],
           -k*x[1] + k*x[0] -k1*x[1] + k1*x[2] ,
           k*x[0] + k*x[1] + k1*x[0] + k1*x[1] - k1*x[2] - k2*x[2],
           k2*x[2],
           k2*x[2])

In [89]:
ts=np.linspace(0, 10, 10)
x0 = [100, 100, 100, 100, 100]
k1=0.1
k2 = 0.05
k=0.2
solx = odeint(dX_dt, x0, ts, args=(k1, k2, k))
print(solx)

[[100.         100.         100.         100.         100.        ]
 [ 79.26500267  99.95870168 142.44869023 106.79928146 106.79928146]
 [ 68.53261528 100.50284085 173.78261478 115.62324    115.62324   ]
 [ 63.78511504 102.10621847 198.36211097 125.98431717 125.98431717]
 [ 62.61951488 104.85199702 218.99548377 137.59100538 137.59100538]
 [ 63.61293038 108.66285995 237.48546114 150.27784584 150.27784584]
 [ 65.93301793 113.41398583 254.98884584 163.96025648 163.96025648]
 [ 69.10051287 118.98403844 272.25108232 178.60563048 178.60563048]
 [ 72.84554388 125.27465738 289.75798823 194.21500532 194.21500532]
 [ 77.02137643 132.21488481 307.83316321 210.81154618 210.81154618]]


Exercice 3

In [11]:
x0_ex3=[100,100,100,100]

#Vr
#           R1 R2 R3 R4 R5 R6
# dna*      -1 0  0  0  0  0
# dna       0  -1 -1 0  0  0
# rna       0  0  0  -1 -1  0
# protein   0  0  0  0   0  -1

#V
#           R1 R2 R3 R4 R5 R6
# dna*      -1 1  0  0  0  0
# dna       1  -1 0  0  0  0
# rna       0  0  1  0  -1 0
# protein   0  0  0  1  0  -1

k_ex3=[k1,k1,k,k,k,k]

Vr_ex3=[[-1,0,0,0,0,0], [0,-1,-1,0,0,0],[0,0,0,-1,-1,0],[0,0,0,0,0,-1]]
V_ex3= [[-1,1,0,0,0,0], [1,-1,0,0,0,0],[0,0,1,0,-1,0],[0,0,0,1,0,-1]]

gillespie(x0_ex3,k_ex3,2,V_ex3,Vr_ex3)

([101, 100, 102, 21],
 matrix([[100.23529412, 100.        , 101.08235294,  60.68235294]]),
 matrix([[ 0.4241825 ,  0.        ,  0.99660323, 22.67582894]]))

Exercice 4

In [12]:
x0_ex4=[100,100]

#V
#         R1   R2   R3
#prey     1    -1   0
#predator 0    1   -1

#Vr
#         R1   R2   R3
#prey     0    -1   0
#predator 0    -1   -1

k_ex4=[k1,k2,k]

V_ex4=[[1,-1,0],[0,1,-1]]
Vr_ex4=[[0,-1,0],[0,-1,-1]]

gillespie(x0_ex4,k_ex4,2,V_ex4,V_ex4)

([99, 22],
 matrix([[99.37037037, 60.19753086]]),
 matrix([[ 0.48290388, 23.07819224]]))

Exercice 5

In [13]:
V_ex5=[[1,0,-1,1,0],[0,1,0,0,-1],[0,0,0,0,1],[0,0,0,0,1]]
Vr_ex5=[[0,0,-1,0,0],[0,0,0,0,-1],[0,0,-1,-2,-1],[0,0,0,0,0]]

k_ex5=[k, k, k1, k1, k2]
x0_ex5=[100, 100, 100, 100]

gillespie(x0_ex5,k_ex5,2,V_ex5,Vr_ex5)

([103, 99, 103, 102],
 matrix([[101.42857143,  99.        , 101.57142857, 100.57142857]]),
 matrix([[1.17803018, 0.75592895, 1.17803018, 0.72843136]]))

Exercice 6

In [14]:
# 0-> S 
# S-> I
# I-> R
# I-> 0 

x0_ex6 = [100,100,100]
#V

#     R1   R2   R3   R4
#  S  1    -1   0    0
#  I  0    1    -1   -1
#  R  0    0    1    0

#Vr

#     R1   R2   R3   R4
#  S  0    -1   0    0
#  I  0    0    -1   -1
#  R  0    0    0    0

V_ex6=[[1,-1,0,0], [0,1,-1,-1],[0,0,1,0]]

Vr_ex6 =[[0,-1,0,0], [0,0,-1,-1],[0,0,0,0]]

k_ex6 = [k1,k1,k2,k]

gillespie(x0_ex6,k_ex6,2,V_ex6,Vr_ex6)

([99, 98, 101],
 matrix([[100.        ,  97.66666667, 100.44444444]]),
 matrix([[0.66666667, 1.15470054, 0.49690399]]))